# Spark SQL

## Overview

In this and the following two sections, we will discuss Spark SQL. This is the aspect of Spark
that allows us to query a _DataFrame_ using SQL syntax. Spark SQL is indeed a core component of Spark as it allows
to integrate relational like processing with the functional API exposed by Spark.

## Spark SQL

In this section,  we will introduce Spark SQL. This is a Spark interface that allows us to use SQL to query the data; see the script below.
The script works with the Iris dataset but feel free to use any dataset you would like. We rename all the
columns of the dataset so that an underscored is used instead of a dash. Otherwise Spark complains not being
able to find colums

```
from pathlib import Path
from pyspark.sql import SparkSession


DATA_PATH = Path("/home/alex/qi3/qi3_notes/computational_mathematics/src/datasets/iris/iris.data")
APP_NAME = "Spark SQL 1"

if __name__ == '__main__':

    # get a spark session
    spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

    schema = ("`sepal-length` FLOAT, `sepal-width` FLOAT, "
              "`petal-length` FLOAT, `petal-width` FLOAT, `class` STRING")

    # load the data
    df = (spark.read.format("csv")
              .option("header", False)
              .option("inferSchema", False)
              .option("delimiter", ",")
              .schema(schema)
              .load(str(DATA_PATH)))

    df.show(n=10)
    df.printSchema()

    # rename the columns
    # we want to rename the column class to class-idx
    new_df = (df.withColumnRenamed("sepal-length", "sepal_length")
              .withColumnRenamed("sepal-width", "sepal_width")
              .withColumnRenamed("petal-length", "petal_length")
              .withColumnRenamed("petal-width", "petal_width"))

    # create a temporary table
    new_df.createOrReplaceTempView("iris_data_tbl")

    # now we can do SQL queries
    query = """SELECT sepal_length, sepal_width, class FROM iris_data_tbl WHERE 
    sepal_length > 2.3 AND sepal_width > 1.3 ORDER BY sepal_width DESC"""
    spark.sql(query).show(n=10)

    # use the CASE construct
    query = """SELECT sepal_length, petal_length, class, CASE WHEN sepal_length < 2.3 THEN 'SMALL' 
    WHEN sepal_length >= 2.3 AND sepal_length < 5.8 THEN 'MEDIUM' 
    WHEN sepal_length >= 5.8 THEN 'LARGE' 
    END AS Sepal_Length_Size FROM iris_data_tbl ORDER BY class DESC"""
    spark.sql(query).show(n=20)
    spark.stop()
```

The script introduces two new methods for us. Namely, ```spark.sql``` and ```df.createOrReplaceTempView```.
The first is applied on a _SparkSession_ instance 
whilst the second one is applied on a _DataFrame_. Runnin the script produces the following

```
+------------+-----------+------------+-----------+-----------+
|sepal-length|sepal-width|petal-length|petal-width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 10 rows

root
 |-- sepal-length: float (nullable = true)
 |-- sepal-width: float (nullable = true)
 |-- petal-length: float (nullable = true)
 |-- petal-width: float (nullable = true)
 |-- class: string (nullable = true)

+------------+-----------+-----------+
|sepal_length|sepal_width|      class|
+------------+-----------+-----------+
|         5.7|        4.4|Iris-setosa|
|         5.5|        4.2|Iris-setosa|
|         5.2|        4.1|Iris-setosa|
|         5.8|        4.0|Iris-setosa|
|         5.4|        3.9|Iris-setosa|
|         5.4|        3.9|Iris-setosa|
|         5.7|        3.8|Iris-setosa|
|         5.1|        3.8|Iris-setosa|
|         5.1|        3.8|Iris-setosa|
|         5.1|        3.8|Iris-setosa|
+------------+-----------+-----------+
only showing top 10 rows

+------------+------------+--------------+-----------------+
|sepal_length|petal_length|         class|Sepal_Length_Size|
+------------+------------+--------------+-----------------+
|         6.3|         6.0|Iris-virginica|            LARGE|
|         6.9|         5.7|Iris-virginica|            LARGE|
|         5.8|         5.1|Iris-virginica|            LARGE|
|         7.1|         5.9|Iris-virginica|            LARGE|
|         6.3|         5.6|Iris-virginica|            LARGE|
|         6.5|         5.8|Iris-virginica|            LARGE|
|         7.6|         6.6|Iris-virginica|            LARGE|
|         4.9|         4.5|Iris-virginica|           MEDIUM|
|         7.3|         6.3|Iris-virginica|            LARGE|
|         6.7|         5.8|Iris-virginica|            LARGE|
|         7.2|         6.1|Iris-virginica|            LARGE|
|         6.5|         5.1|Iris-virginica|            LARGE|
|         6.4|         5.3|Iris-virginica|            LARGE|
|         6.8|         5.5|Iris-virginica|            LARGE|
|         5.7|         5.0|Iris-virginica|           MEDIUM|
|         5.8|         5.1|Iris-virginica|            LARGE|
|         6.4|         5.3|Iris-virginica|            LARGE|
|         6.5|         5.5|Iris-virginica|            LARGE|
|         7.7|         6.7|Iris-virginica|            LARGE|
|         7.7|         6.9|Iris-virginica|            LARGE|
+------------+------------+--------------+-----------------+
only showing top 20 rows

```


### Create a database


Tables will reside in a database. In addition, Spark supports two types of tables; managed and unmanaged.
A managed table means that Soark handles both the data and the metadata. For an unmanged table, Spark only manages
the metadata. The following script, show how to create a database in Spark.


```
spark.sql("CREATE DATABASE iris_db")
spark.sql("Use iris_db")
```

Just like in SQL, we need to instruct Spark about the database we want to use. 

### Create managed table

There are two ways to create a managed table;

- Use SQL
- Wrie a _DataFrame_ as a table

The first approach is illustrated in the script below

```
query ="CREATE TABLE managed_iris_data (sepal_length FLOAT, sepal_width FLOAT, petal_length FLOAT, petal_width FLOAT, class STRING)"
spark.sql(query)
```

This is very similar, if not exactly the same as we do in SQL. Howver, if you have _DataFrame_ you can persist in a table as
simple as issuing

```
df.write.saveAsTable("managed_iris_data")
```

### Create unmanaged table

We can create an unmanaged table from data sources e.g. Parquet and CSV files as follows.

```
query ="CREATE TABLE iris_data (sepal_length FLOAT, sepal_width FLOAT, petal_length FLOAT, petal_width FLOAT, class STRING)
USING csv OPTIONS (PATH '/your/path/to/your/csv_data'"
spark.sql(query)
```

And similarly using _DataFrame_

```
df.write.options("path", "/your/path/to/your/csv_data").saveAsTable("iris_data")
```



## Summary

This section touched upon the Spark SQL API using a simple example. The Spark SQL API allows us to use SQL in order to query
a _DataFrame_. This is done by first creating either  a table or a view. 
Tables in Spark can be managed or unmanged.
The next section discusses Spark Views.

## References

1. Jules S. Damji, Brooke Wenig, Tathagata Das, Deny Lee, _Learning Spark. Lighting-fasts data analytics_, 2nd Edition, O'Reilly.